In [0]:
!pip install python-twitter

     |████████████████████████████████| 71kB 2.3MB/s 


In [0]:
def get_tweets(api=None, screen_name=None):
    timeline = api.GetUserTimeline(screen_name=screen_name, count=200)
    earliest_tweet = min(timeline, key=lambda x: x.id).id
    print("getting tweets before:", earliest_tweet)

    while True:
        tweets = api.GetUserTimeline(
            screen_name=screen_name, max_id=earliest_tweet, count=200
        )
        new_earliest = min(tweets, key=lambda x: x.id).id

        if not tweets or new_earliest == earliest_tweet:
            break
        else:
            earliest_tweet = new_earliest
            print("getting tweets before:", earliest_tweet)
            timeline += tweets

    return timeline


In [0]:
import numpy as np
import twitter, re, datetime, pandas as pd

ACCESS_TOKEN = ""
ACCESS_TOKEN_SECRET = ""
CONSUMER_KEY = ""
CONSUMER_SECRET = ""

twitter_keys = {
    'consumer_key':        CONSUMER_KEY,
    'consumer_secret':     CONSUMER_SECRET,
    'access_token_key':    ACCESS_TOKEN,
    'access_token_secret': ACCESS_TOKEN_SECRET
}

api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret'],
    tweet_mode = 'extended',
    sleep_on_rate_limit=True
)

In [0]:
handleName = "JoeBiden"
timeline = get_tweets(api=api, screen_name=handleName)

getting tweets before: 1246890352025444353
getting tweets before: 1239353763820961792
getting tweets before: 1233198883448082432
getting tweets before: 1226677160103960576
getting tweets before: 1217546999433744386
getting tweets before: 1206338536317865986
getting tweets before: 1197185331138891778
getting tweets before: 1188474466155802624
getting tweets before: 1181291856480604160
getting tweets before: 1172336799688978432
getting tweets before: 1164369400331415557
getting tweets before: 1155914818202865665
getting tweets before: 1146641031892365312
getting tweets before: 1137118382245974020
getting tweets before: 1118986242501283849
getting tweets before: 968213300956880897
getting tweets before: 916066235850416130


In [0]:
userTweets = [tweet._json for tweet in timeline]
print(len(userTweets))
print(userTweets[0])

3265
{'created_at': 'Mon Apr 27 16:15:00 +0000 2020', 'id': 1254806260324749315, 'id_str': '1254806260324749315', 'full_text': 'COVID-19 can hit anyone, anywhere — but it doesn’t affect every community equally. This afternoon, join Senator @KamalaHarris and @SymoneDSanders for a discussion on its disproportionate impact on communities of color.\n\nTune in at 3:30 PM ET: https://t.co/xk0GcSpr50 https://t.co/lzco7XHP53', 'truncated': False, 'display_text_range': [0, 266], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'KamalaHarris', 'name': 'Kamala Harris', 'id': 30354991, 'id_str': '30354991', 'indices': [112, 125]}, {'screen_name': 'SymoneDSanders', 'name': 'Symone D. Sanders', 'id': 2434051392, 'id_str': '2434051392', 'indices': [130, 145]}], 'urls': [{'url': 'https://t.co/xk0GcSpr50', 'expanded_url': 'http://JoeBiden.com/Live', 'display_url': 'JoeBiden.com/Live', 'indices': [243, 266]}], 'media': [{'id': 1254804759502086145, 'id_str': '1254804759502086

In [0]:
userTweets = [d for d in userTweets if 'RT' not in d['full_text'][:3]]
print(len(userTweets))
print(userTweets[0])

3077
{'created_at': 'Mon Apr 27 16:15:00 +0000 2020', 'id': 1254806260324749315, 'id_str': '1254806260324749315', 'full_text': 'COVID-19 can hit anyone, anywhere — but it doesn’t affect every community equally. This afternoon, join Senator @KamalaHarris and @SymoneDSanders for a discussion on its disproportionate impact on communities of color.\n\nTune in at 3:30 PM ET: https://t.co/xk0GcSpr50 https://t.co/lzco7XHP53', 'truncated': False, 'display_text_range': [0, 266], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'KamalaHarris', 'name': 'Kamala Harris', 'id': 30354991, 'id_str': '30354991', 'indices': [112, 125]}, {'screen_name': 'SymoneDSanders', 'name': 'Symone D. Sanders', 'id': 2434051392, 'id_str': '2434051392', 'indices': [130, 145]}], 'urls': [{'url': 'https://t.co/xk0GcSpr50', 'expanded_url': 'http://JoeBiden.com/Live', 'display_url': 'JoeBiden.com/Live', 'indices': [243, 266]}], 'media': [{'id': 1254804759502086145, 'id_str': '1254804759502086

In [0]:
def clean(dict_list, to_change, change_to):
    for d in dict_list:
      for k,v in d.items():
        if k == 'full_text':
          # v = ' '.join(v) use if its a list for some stupid fucking reason
          v = v.replace('#', '') # no more hash tag characters
          v = v.replace('@', '') # no more twitter linking characters
          v = v.split(' ') 
          v = filter(lambda x: 'http' not in x, v) # get rid of https links
          v = list(v)
          d[k] = ' '.join(v)
    return dict_list

userTweets = clean(userTweets, '@', '')
print(len(userTweets))

3077


In [0]:
import pandas as pd

In [0]:
userTweetsDf = pd.DataFrame(userTweets, columns=['id_str', 'created_at', 'full_text', 'retweet_count', 'favorite_count', 'lang'])
userTweetsDf.insert(1, 'handle', value=handleName)

In [0]:
userTweetsDf.shape

(3077, 7)

In [0]:
userTweetsDfReduced = pd.DataFrame(userTweetsDf, columns=['id_str', 'created_at', 'full_text', 'retweet_count', 'favorite_count', 'lang'])
userTweetsDfReduced.insert(1, 'handle', value=handleName)

In [0]:
userTweetsDfReduced = userTweetsDfReduced[~userTweetsDfReduced['full_text'].str.contains('RT')]
userTweetsDfReduced = userTweetsDfReduced[userTweetsDfReduced['lang'] == 'en']
userTweetsDfReduced.drop(labels=['lang'], axis=1, inplace=True)
print(userTweetsDfReduced.shape)

(3030, 6)


In [0]:
userTweetsDfReduced.to_csv(handleName + 'tweets.csv', encoding='utf-8', index=False)

In [0]:
df = pd.read_csv(handleName + 'tweets.csv')

In [0]:
df.head()

,id_str,handle,created_at,full_text,retweet_count,favorite_count
0,1254806260324749315,JoeBiden,Mon Apr 27 16:15:00 +0000 2020,"COVID-19 can hit anyone, anywhere — but it doe...",1595,5123
1,1254782856074637319,JoeBiden,Mon Apr 27 14:42:00 +0000 2020,We want our country to get moving and healthy ...,3404,14027
2,1254756132746379266,JoeBiden,Mon Apr 27 12:55:48 +0000 2020,I'm incredibly honored by SpeakerPelosi's endo...,7296,36031
3,1254560894576623616,JoeBiden,Mon Apr 27 00:00:00 +0000 2020,America has always met every big challenge wit...,7243,38172
4,1254538243996921856,JoeBiden,Sun Apr 26 22:30:00 +0000 2020,"Wealthy firms with ties to Trump got millions,...",13189,34769
